### Instructions
1. Click on cell with 'versioninfo()' command and press Ctrl + F8
2. Reload page when its done running the cells
3. Click on cell with 'versioninfo()' command again and press Ctrl + F10 to run it and all remaining cells

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 1 and 2.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime.
* Changes in files from the repository must be done locally through VScode. The change is then commited and synced. Afterwards, the cell "Update repository" must be ran

Get dataset (~30 min)

In [ ]:
%%shell
rm -r sample_data
URL="https://zenodo.org/record/8191138/files/datasets.zip?download=1"
wget -nv $URL -O /tmp/datasets.zip
sudo apt-get install unzip
unzip /tmp/datasets.zip -d /content
rm /tmp/datasets.zip

### Install Julia

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.9.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
fi

### System info

In [ ]:
versioninfo()

Clone/Update repository

In [ ]:
using LibGit2
rm("./QuickTO"; force = true, recursive = true)
repo = LibGit2.clone("https://github.com/LucasMSpereira/QuickTO", "./QuickTO"; branch = "colab")
println("Current commit: ", LibGit2.head("./QuickTO")[1:8])

### Definitions and packages
(~30min)

In [ ]:
const runningInColab = true
# Boolean indicating if packages are already installed. If the
# runtime has recently been (re)started, they aren't.
packagesInstalled::Bool = false
include("./QuickTO/utilsColab.jl")

Parameters

In [ ]:
const batchSize = 64
const normalizeDataset = false # choose to normalize data in [-1; 1]
const startTime = timeNow()
const percentageDataset = 0.01 # fraction of dataset to be used
const wasserstein = true; # use wasserstein loss

**Estimate time** needed to finish training, validating, and testing. This varies with hardware, and supposes GPU usage. Without GPU access, training takes significantly (~6x) longer. The inputs are:

1.   Number of training epochs.
2.   Percentage of dataset used.
3.   Number of training epochs between validations.

After all training and validation epochs, a test epoch is done.

In [ ]:
trainStats(3, percentageDataset, 1)

### Main script

In [ ]:
GC.gc() # garbage collector

# Start straining with specific configuration. After a certain number of training
# epochs, a validation epoch is done. After all training and validation, a test
# epoch is performed. Model checkpoints are occasionally saved. The final models
# are saved as well.
@time expMetaData = trainGANs(;
  ## Choose optimizers for generator and discriminator.
  genOpt_ = Flux.Optimise.AdamW(1e-4),
  discOpt_ = Flux.Optimiser(Flux.ClipNorm(1.0), Flux.Optimise.AdamW(1e-4)),
  ## (optional) If not training from scratch, uncomment lines of next 4
  ## keyword arguments, and provide names of pre-trained models, alongside
  ## the paths to the metadata file and the original folder of the model.
  # genName_ = "01-30T13-49-30-3gen.bson",
  # discName_ = "01-30T13-50-04-3disc.bson",
  # metaDataName = projPath * "networks/GANplots/01-29T09-45-03-Bvp4/01-29T20-07-39metaData.txt",
  # originalFolder = projPath * "networks/GANplots/01-29T09-45-03-Bvp4/",
  ## Determine architectures to be used for each network. Their definitions
  ## are in "./QuickTO/utilities/ML utils/architectures.jl".
  architectures = (
    convNextModel(96, [3, 3, 9, 3], 0.5),
    # convNextModel(128, [3, 3, 27, 3], 0.5),
    # convNextModel(192, [3, 3, 27, 3], 0.5),
    # U_SE_ResNetGenerator(),
    # patchGANdisc()
    topologyGANdisc()
  ),
  ## Define training configurations, only total number of epochs,
  ## and validation interval are required. Definition is
  ## in ./QuickTO/utilities/typeDefinitions.jl
  trainConfig = epochTrainConfig(3, 1)
  # trainConfig = earlyStopTrainConfig(
  #   1; earlyStopQuant = 2, earlyStopPercent = 5
  # )
)
saveGANs(expMetaData, 0; finalSave = true) # save final models
switchTraining(expMetaData, false)